In [1]:
# Import the required packages
# Figures
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl

# Ipython notebook basics
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import sys
sys.setrecursionlimit(5000)

# Things I use everytime
import numpy as np
np.random.seed(42)
import pandas as pd
#import scipy.stats as stats

import statsmodels.api as sm
import re
import seaborn as sns

import scipy.stats as stats

In [2]:
# Parameter tagged cell for papermill
filename = "default"
#iso_seq_gene = pd.read_csv(f"/mmfs1/gscratch/stergachislab/yhhc/projects/Iso-seq_public/Cyclo_noncyclo_comparison/Analysis/6.10.24/3.Compare_samples/3.Novel_iso_abundance_in_gene/data_combined_full_gene_with_Hyp5.csv",sep=",")
#iso_seq_gene = pd.read_csv(f"/mmfs1/gscratch/stergachislab/yhhc/projects/Iso-seq_public/Cyclo_noncyclo_comparison/Analysis/6.10.24/3.Compare_samples/1.Isoform/data_combined_full.csv",sep=",")


In [3]:
iso_seq_gene = pd.read_csv(filename,sep=",")

FileNotFoundError: [Errno 2] No such file or directory: 'default'

In [ ]:
# Remove isoforms with low counts in all patients.

# Define count threshold
count_threshold = 10

# Group by 'Isoform_PBid' and check if any counts meet the threshold
isoforms_to_keep = iso_seq_gene.groupby('Isoform_PBid').apply(
    lambda group: any(group['cyclo_count'] >= count_threshold) or any(group['noncyclo_count'] >= count_threshold)
)

# Filter to keep only isoforms that meet the threshold
isoforms_to_keep = isoforms_to_keep[isoforms_to_keep].index.tolist()

# Print helpful statements
print("Number of isoforms to keep after filtering based on counts:")
print(len(isoforms_to_keep))

# Filter the data_with_totals to keep only the desired Isoform_PBid
iso_seq_gene = iso_seq_gene[iso_seq_gene['Isoform_PBid'].isin(isoforms_to_keep)]


In [ ]:
# Add prop_novel
iso_seq_gene['prop_novel'] = (iso_seq_gene['proportion_in_Bin1_cyclo'] - iso_seq_gene['proportion_in_Bin1_noncyclo']) / (iso_seq_gene['proportion_in_Bin1_cyclo'] + iso_seq_gene['proportion_in_Bin1_noncyclo'])

In [ ]:
# In the iso_seq_gene pd dataframe, I have a column called Cyclo_TPM and another column called Noncyclo_TPM.
# I want to create a column called test_statistic that is calculated by doing this:
# ratio = (Cyclo_TPM+1)/(Noncyclo_TPM+1)
# absolute_diff = abs(Cyclo_TPM - Noncyclo_TPM)
# test_statistic = log2(ratio) * log2(5*absolute_diff)

df = iso_seq_gene

# Calculate the adjusted ratio
df['ratio_cyclo'] = (df['Total_bin_cyclo_count_Bin1_le']) / (df['cyclo_count']+ 1)
df['ratio_noncyclo'] = (df['Total_bin_noncyclo_count_Bin1_le']) / (df['noncyclo_count'] + 1)
#df['ratio'] = (df['ratio_cyclo'] + 0.001) / (df['ratio_noncyclo'] + 0.001)

df['test_statistic'] = (
    (df['ratio_cyclo'] - df['ratio_noncyclo'])
    * np.log2(df['Total_bin_cyclo_count_Bin1_le'] + 1) 
    * np.log2(df['Total_bin_noncyclo_count_Bin2_g'] + 1)
)

#df['test_statistic'] = (df['ratio_cyclo']-df['ratio_noncyclo']) * np.log2(df['Total_bin_cyclo_count_Bin1_le_TPM'] + 1) * np.log2(df['Total_bin_noncyclo_count_Bin2_g_TPM'] + 1) 


# # V1: 
# # Using TPM improves the ranking for MFN2. It also decreases the skewness for the over-sequenced samples. 
# df['Total_bin_cyclo_count_Bin1_le_TPM'] = (df['Total_bin_cyclo_count_Bin1_le'] / df['cyclo_total_sum']) * 1000000
# df['test_statistic'] = (df['ratio_cyclo']-df['ratio_noncyclo']) * np.log2(df['Total_bin_cyclo_count_Bin1_le_TPM'] + 1) #This works fine.

In [ ]:
# Calculate z-score, compares patient to other patients.

# # Function to calculate the z-score for each row within a group
# def calculate_z_score_within_group(group):
#     group = group.copy()  # Avoid SettingWithCopyWarning
#     for i, row in group.iterrows():
#         other_test_statistics = group.loc[group.index != i, 'test_statistic']
#         mean_others = other_test_statistics.mean()
#         sd_all = group['test_statistic'].std()
#         group.at[i, 'z_score'] = (row['test_statistic'] - mean_others) / sd_all
#     return group

# Function to calculate the z-score for each row within a group
def calculate_z_score_within_group(group):
    group = group.copy()  # Avoid SettingWithCopyWarning
    for i, row in group.iterrows():
        other_test_statistics = group.loc[group.index != i, 'test_statistic']
        median_others = other_test_statistics.median()
        sd_all = group['test_statistic'].std()

        if sd_all == 0:
            group['z_score'] = 0
        else:
            group.at[i, 'z_score'] = (row['test_statistic'] - median_others) / sd_all
            # Can't use IQR because iqr is 0 for the HARS1 positive control case and causes division by zero. 
    
    return group

# Apply the function to each group
df = df.groupby('Isoform_PBid').apply(calculate_z_score_within_group)

# Reset index to clean up the DataFrame
df.reset_index(drop=True, inplace=True)


#[T1 - mean(T2,...,T18)]/SD(T1,....,T18)

## Histogram of test statistic

In [ ]:
# Drop NaN values from the test_statistic column for plotting
test_statistic_values = df['test_statistic'].dropna()

# Create a histogram of the test_statistic values
plt.hist(test_statistic_values, bins=100, edgecolor='black')
plt.xlabel('Test Statistic')
plt.ylabel('Frequency')
plt.title('Histogram of Test Statistic Values')
plt.show()

In [ ]:
plt.hist(test_statistic_values, bins=100, edgecolor='black')
plt.xlabel('Test Statistic')
plt.ylabel('Frequency')
plt.title('Zoomed in Histogram of Test Statistic Values')
plt.xlim(-3, 3)  # Set x-axis range from -25 to 25
plt.show()

## Summary stats of test statistic

In [ ]:
# Drop NaN values from the test_statistic column
test_statistic_values = df['test_statistic'].dropna() # NA occurs when the abs diff is 0 and you take log2(0)

# Get summary statistics
summary_stats = test_statistic_values.describe()
summary_stats

In [ ]:
# # The amount of NAs should equal the amount of rows with abs diff of 0. LEFT OFF HERE
# num_na_values = df['test_statistic'].isna().sum()

# # Count the number of rows where absolute_diff is zero
# num_abs_diff_zero = (df['absolute_diff'] == 0).sum()

# print("\nNumber of NaN values in test_statistic:", num_na_values)
# print("Number of rows with absolute_diff equal to 0:", num_abs_diff_zero)

In [ ]:
# Get the row with the max test_statistic
max_test_statistic_row = df.loc[df['test_statistic'].idxmax()]

# Get the row with the min test_statistic
min_test_statistic_row = df.loc[df['test_statistic'].idxmin()]

max_min_test_statistic_df = pd.DataFrame([max_test_statistic_row, min_test_statistic_row])

# Set display options to show all columns
pd.set_option('display.max_columns', None)

max_min_test_statistic_df

# Reset to default after displaying
pd.reset_option('display.max_columns')

## Correlation of test statistic with P_Value_Hyp5

In [ ]:

# # Calculate the negative log of P_Value_Hyp1
# df['neg_log_P_Value_Hyp5'] = -np.log10(df['P_Value_Hyp5'])

# # Filter the DataFrame to only include rows where NormalizedFractionDifference > 0
# filtered_df = df[df['prop_novel'] > 0]

# # Drop NaN values from the test_statistic and neg_log_P_Value_Hyp1 columns for correlation analysis
# filtered_df = filtered_df.dropna(subset=['test_statistic', 'neg_log_P_Value_Hyp5'])

# # Calculate the correlation value
# correlation_value = filtered_df[['test_statistic', 'neg_log_P_Value_Hyp5']].corr().loc['test_statistic', 'neg_log_P_Value_Hyp5']

# # Create the scatter plot
# plt.scatter(filtered_df['test_statistic'], filtered_df['neg_log_P_Value_Hyp5'], alpha=0.5)
# plt.xlabel('Test Statistic')
# plt.ylabel('-log10(P_Value_Hyp5)')
# plt.title(f'Scatter Plot of Test Statistic with prop_novel >0 vs. -log10(P_Value_Hyp5)\nCorrelation: {correlation_value:.2f}')
# plt.grid(True)
# plt.show()

## Histogram of z-scores

In [ ]:
# Drop NaN values from the test_statistic column for plotting
z_score_values = df['z_score'].dropna()

# Create a histogram of the test_statistic values
plt.hist(z_score_values, bins=100, edgecolor='black')
plt.xlabel('Z-Score')
plt.ylabel('Frequency')
plt.title('Histogram of Z-Score Values')
plt.show()

In [ ]:
z_score_values = df['z_score'].dropna()
z_score_values = z_score_values[(z_score_values >= -50) & (z_score_values <= 50)]


plt.hist(z_score_values, bins=100, edgecolor='black')
plt.xlabel('Z-Score')
plt.ylabel('Frequency')
plt.title('Zoomed in Histogram of Z-score Values')
plt.xlim(-25, 25)  # Set x-axis range from -25 to 25
plt.show()

## Summary stats of z-scores

In [ ]:
# Drop NaN values from the test_statistic column
z_score_values = df['z_score'].dropna() # NA occurs when the abs diff is 0 and you take log2(0)

# Get summary statistics
summary_stats_z_score = z_score_values.describe()
summary_stats_z_score

In [ ]:
# Get the row with the max test_statistic
max_z_score_row = df.loc[df['z_score'].idxmax()]

# Get the row with the min test_statistic
min_z_score_row = df.loc[df['z_score'].idxmin()]

max_min_z_score_df = pd.DataFrame([max_z_score_row, min_z_score_row])

# Set display options to show all columns
pd.set_option('display.max_columns', None)

max_min_z_score_df

# Reset to default after displaying
pd.reset_option('display.max_columns')

## Correlation of z-score with P_Value_Hyp5

In [ ]:

# # Calculate the negative log of P_Value_Hyp1
# df['neg_log_P_Value_Hyp5'] = -np.log10(df['P_Value_Hyp5'])

# # Filter the DataFrame to only include rows where NormalizedFractionDifference > 0
# filtered_df = df[df['prop_novel'] > 0]

# # Drop NaN values from the test_statistic and neg_log_P_Value_Hyp1 columns for correlation analysis
# filtered_df = filtered_df.dropna(subset=['z_score', 'neg_log_P_Value_Hyp5'])

# # Calculate the correlation value
# correlation_value = filtered_df[['z_score', 'neg_log_P_Value_Hyp5']].corr().loc['z_score', 'neg_log_P_Value_Hyp5']

# # Create the scatter plot
# plt.scatter(filtered_df['z_score'], filtered_df['neg_log_P_Value_Hyp5'], alpha=0.5)
# plt.xlabel('z_score')
# plt.ylabel('-log10(P_Value_Hyp5)')
# plt.title(f'Scatter Plot of Z-Score with prop_novel >0 vs. -log10(P_Value_Hyp5)\nCorrelation: {correlation_value:.2f}')
# plt.grid(True)
# plt.show()

## Write outputs

In [ ]:
df.to_csv('test_statistic_hyp5.csv', index=False)

In [ ]:
import pickle

# Function to save variables
def save_variables(filename, variables):
    with open(filename, 'wb') as f:
        pickle.dump(variables, f)

# Save specific variables
variables_to_save = {
    'df': df
}
save_variables('Test_statistic_hyp5_notebook_variables.pkl', variables_to_save)

# Load variables from a file
# def load_variables(filename):
#     with open(filename, 'rb') as f:
#         variables = pickle.load(f)
#     return variables

# # Load the variables
# loaded_variables = load_variables('notebook_variables.pkl')

# # Access the loaded variables
# df = loaded_variables['df']

## Positive controls

In [ ]:
# Display all the positive control cases.

#UDN212054 HARS1
#UDN633333 MFN2
#UDN215640 SET

# Define the pairs to match
matching_pairs = {
    'UDN212054': 'HARS1',
    'UDN633333': 'MFN2',
    'UDN215640': 'SET'
}

# Filter the DataFrame based on the matching pairs
filtered_df = df[df.apply(lambda row: matching_pairs.get(row['Sample']) == row['associated_gene'], axis=1)]

# Set display options to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#data_df[data_df['associated_gene'] == 'MFN2']
filtered_df

# Reset to default after displaying
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')



## SRSF6 cassette exon positive control

In [ ]:
# Set display options to show all columns
pd.set_option('display.max_columns', None)

df[df['associated_gene'] == 'SRSF6']

# Reset to default after displaying
pd.reset_option('display.max_columns')


## Histogram of Z-score in a single patient

In [ ]:
# Get unique samples
unique_samples = df['Sample'].unique()

# Create subplots
num_samples = len(unique_samples)
num_cols = 3
num_rows = (num_samples + num_cols - 1) // num_cols

fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, num_rows * 5))

# Flatten the axes array for easy iteration
axes = axes.flatten()

for i, sample in enumerate(unique_samples):
    filtered_df = df[df['Sample'] == sample]
    z_score_values = filtered_df['z_score'].dropna()
    
    # Create histogram in the appropriate subplot
    axes[i].hist(z_score_values, bins=100, edgecolor='black')
    axes[i].set_xlabel('Z-Score')
    axes[i].set_ylabel('Frequency')
    axes[i].set_title(f'Histogram of Z-Score Values for Sample {sample}')

# Remove any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

## Z-score vs test-statistic

In [ ]:
# Filter the DataFrame to include only rows for the sample 'UDN212054'
filtered_df = df[df['Sample'] == 'UDN633333']

# Create a scatter plot with different colors
colors = ['red' if gene == 'MFN2' else 'blue' for gene in filtered_df['associated_gene']]
plt.scatter(filtered_df['test_statistic'], filtered_df['z_score'], c=colors, edgecolor='black')

# Label points where associated_gene is 'HARS1'
for i, row in filtered_df.iterrows():
    if row['associated_gene'] == 'MFN2':
        plt.annotate('MFN2', (row['test_statistic'], row['z_score']),
                     textcoords="offset points", xytext=(5,5), ha='center', color='red')

plt.xlabel('Test Statistic')
plt.ylabel('Z-Score')
plt.title('Scatter Plot of Z-Score vs Test Statistic for Sample UDN633333')
plt.show()

## Plotting prop_novel against Z-score

In [ ]:
# Filter the DataFrame to include only rows for the sample 'UDN212054'
filtered_df = df[df['Sample'] == 'UDN633333']

# Create a scatter plot
colors = ['red' if gene == 'MFN2' else 'blue' for gene in filtered_df['associated_gene']]
plt.scatter(filtered_df['prop_novel'], filtered_df['z_score'], c=colors, edgecolor='black')

# Label points where associated_gene is 'HARS1'
for i, row in filtered_df.iterrows():
    if row['associated_gene'] == 'MFN2':
        plt.annotate('MFN2', (row['prop_novel'], row['z_score']),
                     textcoords="offset points", xytext=(5,5), ha='center', color='red')

plt.xlabel('prop_novel')
plt.ylabel('Z-Score')
plt.title('Scatter Plot of Z-Score vs prop_novel for Sample UDN633333')
plt.show()

In [ ]:
# Find the maximum z-score value
max_z_score = df['z_score'].max()

# Filter the DataFrame to include only rows with the maximum z-score
max_z_score_rows = df[df['z_score'] == max_z_score]

# Calculate the number of rows with the maximum z-score
num_max_z_score_rows = len(max_z_score_rows)

print(f"The number of rows with the maximum z-score is: {num_max_z_score_rows}")

In [ ]:
max_z_score_rows

## Rank of positive control

In [ ]:
# Define the percentiles to check
percentiles = [99.9, 99.5, 99, 98, 95, 90, 80, 50]

# Filter the DataFrame to include only rows for the sample 'UDN212054'
sample_df = df[df['Sample'] == 'UDN633333']
sample_df = sample_df[sample_df['prop_novel'] > 0]


# Loop through each percentile, calculate and print the rank
for percentile in percentiles:
    # Calculate the percentile of the z-scores within this sample
    percentile_value = np.percentile(sample_df['z_score'], percentile)

    
    # Filter the DataFrame to include only rows with z-scores above this percentile within the sample
    top_percentile_df = sample_df[sample_df['z_score'] > percentile_value]
    
    # Sort the filtered DataFrame by the test statistic in descending order
    sorted_df = top_percentile_df.sort_values(by='test_statistic', ascending=False)
    
    # Reset the index to create a ranking
    sorted_df = sorted_df.reset_index(drop=True)
    
    try:
        # Determine the rank of 'HARS1' in the sorted DataFrame
        hars1_rank = sorted_df[sorted_df['associated_gene'] == 'MFN2'].index[0] + 1  # Adding 1 to convert zero-based index to rank
        print(f"The rank of MFN2 in terms of test statistic among the top {percentile} percentile of z-scores in UDN633333 is: {hars1_rank}")
    except IndexError:
        print(f"MFN2 is not in the top {percentile} percentile of z-scores in UDN633333.")

In [ ]:
# Set the display option to show all columns
pd.set_option('display.max_columns', None)

sorted_df.head(22)

# Reset the display option to its default value
pd.reset_option('display.max_columns')


## Volcano plot replacement

In [ ]:
## For the top 5% of isoforms in UDN212054 based on z-score, plot the NormalizedFractionDifference vs test_statistic. 
# Label and color red the HARS1 isoforms.
# Filter the DataFrame to include only rows for the sample 'UDN212054'
sample_df = df[df['Sample'] == 'UDN633333']
sample_df = sample_df[sample_df['prop_novel'] > 0]

# Calculate the 95th percentile of the z-scores within this sample
percentile_95 = np.percentile(sample_df['z_score'], 99)

# Filter the DataFrame to include only the top 5% isoforms based on z_score
top_5_percent_df = sample_df[sample_df['z_score'] > percentile_95]

# Plot NormalizedFractionDifference vs test_statistic
plt.figure(figsize=(10, 6))
plt.scatter(top_5_percent_df['prop_novel'], top_5_percent_df['test_statistic'], color='blue', label='Isoforms')

# Highlight and label HARS1 isoforms
for i, row in top_5_percent_df.iterrows():
    if row['associated_gene'] == 'MFN2':
        plt.scatter(row['prop_novel'], row['test_statistic'], color='red', label='MFN2')
        plt.annotate('MFN2', (row['prop_novel'], row['test_statistic']),
                     textcoords="offset points", xytext=(0,10), ha='center', color='red')

plt.xlabel('prop_novel')
plt.ylabel('Test Statistic')
plt.title('prop_novel vs Test Statistic for Top 1% Genes in UDN633333 based on z-score')
plt.legend(loc='best')
plt.show()

In [ ]:
# Delete all variables
%reset -f

# Additional cleanup if needed
import gc
gc.collect()